In [1]:
# import argparse
import glob
import os
import sys
import numpy as np
import pandas as pd
import skimage.io as skimg_io
from sklearn.metrics import accuracy_score
from d3mds import D3MDS

import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
def load_imgs_from_files(img_files, img_dir):
    X = []
    for i, img_file in enumerate(img_files):
        if i % 1000 == 0:
            print('Loading Image %d' % i)
        img = skimg_io.imread(os.path.join(img_dir, img_file))        
        X.append(img)
    X = np.stack(X, axis=0)
    X = X[:, :, :, None]
    return X

In [3]:
def build_model(num_classes=10, input_size=16):
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1),
                     activation='relu',
                     input_shape=(input_size, input_size, 1)))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(200, activation='relu'))
    model.add(Dense(num_classes))#, activation='softmax'))
    model.add(Activation(tf.nn.softmax))
    
    return model

In [4]:
def evaluate_on_test_set(model, X_test, y_test):
    preds = np.argmax(model.predict(X_test), axis=1)
    preds += 1
    acc_test = accuracy_score(y_test, preds)
    print('Test Accuracy: %.2f' % (acc_test * 100))
    return acc_test, preds

In [5]:
def write_predictions_csv_file(inds, preds, prediction_filename):     
    df = pd.DataFrame(preds, index=inds, columns=['label'])
    df.to_csv(prediction_filename, index_label='d3mIndex')

def write_scores_csv_file(metric_dict, score_filename):    
    metric_names = []
    metric_values = []
    for metric_name, metric_value in metric_dict.items():
        metric_names.append(metric_name)
        metric_values.append(metric_value)
    metric_names = np.array(metric_names)
    metric_values = np.array(metric_values)

    df = pd.DataFrame(np.concatenate((metric_names[:, None], metric_values[:, None]), axis=1), columns=['metric', 'value'])
    df.to_csv(score_filename)

In [6]:
dataset_root = '../../'
model_dir = './models/'

print('Load DATA')    
data_path = glob.glob(os.path.join(dataset_root, "*_dataset"))[0]
problem_path = glob.glob(os.path.join(dataset_root, "*_problem"))[0]
d3mds = D3MDS(data_path, problem_path)

Load DATA


In [7]:
print('\nLoad train data')
df_train = d3mds.get_train_data()
media_dir = os.path.join(data_path, 'media')
X_train = load_imgs_from_files(df_train['image'], media_dir)
y_train = d3mds.get_train_targets()
X_train = X_train.astype('float32') / 255.0
print(X_train.shape, y_train.shape)


Load train data
Loading Image 0
Loading Image 1000
Loading Image 2000
Loading Image 3000
Loading Image 4000
Loading Image 5000
Loading Image 6000
Loading Image 7000
(7291, 16, 16, 1) (7291, 1)


In [8]:
print('Load test data')
df_test = d3mds.get_test_data()
X_test = load_imgs_from_files(df_test['image'], media_dir)
y_test = d3mds.get_test_targets()
X_test = X_test.astype('float32') / 255.0
print(X_test.shape, y_test.shape)

Load test data
Loading Image 0
Loading Image 1000
Loading Image 2000
(2007, 16, 16, 1) (2007, 1)


In [9]:
num_classes = len(np.unique(y_train))
model = build_model(num_classes=num_classes, input_size=X_train.shape[1])
model.summary()

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.SGD(lr=0.01),
              metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 12, 12, 32)        832       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 64)          18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 2, 2, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 200)               51400     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                2010      
__________

In [10]:
y_train_temp = y_train - 1
y_train_one_hot = to_categorical(y_train_temp, num_classes=num_classes)

batch_size = 128
epochs = 10

model_file = 'model_weights.h5'
model_full_path = os.path.join(model_dir, model_file)
if os.path.exists(model_full_path):
    print('Loading model from %s' % model_full_path)
    model.load_weights(model_full_path)
else:
    print('Training model')
    model.fit(X_train, y_train_one_hot,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1)
    model.save(model_full_path)

print('Performance on TEST sest')
acc_test, preds_test = evaluate_on_test_set(model, X_test, y_test)

Training model
Epoch 1/10
7291/7291 [==============================] - 1s 180us/step - loss: 2.2521 - acc: 0.2301
Epoch 2/10
7291/7291 [==============================] - 1s 129us/step - loss: 2.1654 - acc: 0.3004
Epoch 3/10
7291/7291 [==============================] - 1s 135us/step - loss: 2.0174 - acc: 0.3322
Epoch 4/10
7291/7291 [==============================] - 1s 130us/step - loss: 1.7440 - acc: 0.4578
Epoch 5/10
7291/7291 [==============================] - 1s 134us/step - loss: 1.3662 - acc: 0.6762
Epoch 6/10
7291/7291 [==============================] - 1s 126us/step - loss: 1.0107 - acc: 0.7895
Epoch 7/10
7291/7291 [==============================] - 1s 131us/step - loss: 0.7632 - acc: 0.8299
Epoch 8/10
7291/7291 [==============================] - 1s 134us/step - loss: 0.6136 - acc: 0.8494
Epoch 9/10
7291/7291 [==============================] - 1s 131us/step - loss: 0.5266 - acc: 0.8611
Epoch 10/10
7291/7291 [==============================] - 1s 137us/step - loss: 0.4728 - acc: 0

In [11]:
print('Writing predictions to .csv file.')
cur_dir = './'
predictions_file = os.path.join(cur_dir, 'predictions.csv')
write_predictions_csv_file(df_test.index, preds_test, predictions_file)

print('Writing scores to .csv file.')
metric_dict = {'accuracy': acc_test}
scores_file = os.path.join(cur_dir, 'scores.csv')
write_scores_csv_file(metric_dict, scores_file)

Writing predictions to .csv file.
Writing scores to .csv file.
